In [1]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import math
import numpy as np
import ast
from tqdm import tqdm
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score

In [2]:
transactions = pd.read_csv("transactions.csv")
transactions.sort_values(by="purchase_date", ascending=False)

# Convert the string date to datetime format.
transactions['purchase_date'] = pd.to_datetime(transactions['purchase_date'])
transactions["day"] = (transactions["purchase_date"] - datetime(2020, 6, 1)).apply(lambda x: x.days)
transactions

,customer_id,product_id,purchase_date,quantity,day
0,38769,3477,2020-06-01,1,0
1,42535,30474,2020-06-01,1,0
2,42535,15833,2020-06-01,1,0
3,42535,20131,2020-06-01,1,0
4,42535,4325,2020-06-01,1,0
...,...,...,...,...,...
1071533,22114,29567,2021-01-31,2,244
1071534,22114,21068,2021-01-31,1,244
1071535,10900,18270,2021-01-31,1,244
1071536,17894,31265,2021-01-31,5,244


In [3]:
products = pd.read_csv("product_catalog.csv")
products.sort_values(by="product_id",inplace=True)
products.reset_index(drop=True,inplace=True)

products["categories"] = products["categories"].fillna("[3898]")
products["categories"] = products["categories"].apply(ast.literal_eval)

# Attributes that have the value -1 is filled up according to attribute_2's mode value.
# For instance, since attribute_1 most often takes the value 4 when attribute_2 is 0, the value -1 of attribute_1 is replaced with 4
# and when attribute_2 is 2, attribute_5 is most often takes 66.
# So, where the attribute_2 is 2 and attribute_5 is -1, the attribute_5's -1 value is replaced with 66.
#We choose attribute_2 because it doesn't contain any -1, so it is the most correct data we currently have.

attribute_columns = ['attribute_1', 'attribute_3', 'attribute_4', 'attribute_5']
modes = products.groupby(by="attribute_2")[attribute_columns].apply(lambda x: x[x != -1].mode().iloc[0])
for col in attribute_columns:
    products[col] = products.apply(lambda row: int(modes.loc[row['attribute_2']][col]) if row[col] == -1 else int(row[col]),axis=1)

products

,product_id,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,categories
0,0,389,4,2,469,0,66,[3898]
1,1,1006,4,0,487,3,66,"[2459, 274, 4124]"
2,2,464,4,1,484,0,66,[3898]
3,3,389,8,1,529,0,39,"[807, 3403]"
4,4,1291,4,0,457,0,17,"[281, 1114, 3800]"
...,...,...,...,...,...,...,...,...
32771,32771,995,10,0,515,3,188,[3898]
32772,32772,333,6,1,503,0,17,"[2005, 1000, 3941]"
32773,32773,284,4,1,531,0,148,"[1705, 385, 2256, 2722, 248, 346, 3042]"
32774,32774,433,4,0,455,0,17,[3254]


In [4]:
product_categories = pd.read_csv("product_category_map.csv")
product_categories = product_categories.drop_duplicates("category_id").reset_index(drop=True)
display(product_categories)

#We checked each parent chain to detect if there are any cycles.
# We discovered that 3898 actually is the main parent_category_id of all category_ids. All parent_category_ids are end up with 3898.
hierarchy_chains = []
for cat in product_categories["category_id"]:
    lst = []
    while True:
        lst.append(cat)
        cat = product_categories.loc[product_categories["category_id"]== cat,"parent_category_id"].values[0]
        if cat in lst:
            hierarchy_chains.append(lst)
            break

,category_id,parent_category_id
0,0,75
1,1,1499
2,2,1082
3,3,3498
4,4,1623
...,...,...
4295,4295,3898
4296,4296,3898
4297,4297,3898
4298,4298,3898


In [17]:
for i in range(len(products)):
    for j in range(len(products.loc[i, "categories"])):
        cat = products.loc[i, "categories"][j]
        if len(hierarchy_chains[cat]) > 1:
            products.loc[i,"categories"][j] = hierarchy_chains[cat][-2]

products

,product_id,manufacturer_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,categories
0,0,389,4,2,469,0,66,[3898]
1,1,1006,4,0,487,3,66,"[1840, 2920, 4124]"
2,2,464,4,1,484,0,66,[3898]
3,3,389,8,1,529,0,39,"[2346, 2346]"
4,4,1291,4,0,457,0,17,"[2006, 2006, 3616]"
...,...,...,...,...,...,...,...,...
32771,32771,995,10,0,515,3,188,[3898]
32772,32772,333,6,1,503,0,17,"[3056, 1559, 3941]"
32773,32773,284,4,1,531,0,148,"[370, 370, 370, 2346, 686, 2346, 370]"
32774,32774,433,4,0,455,0,17,[366]


In [5]:

test_set = pd.read_csv("test.csv")
test_set = test_set.set_index("id")
test_set["prediction"] = 0
test_set

,customer_id,product_id,prediction
id,,,
0,0,20664,0
1,0,28231,0
2,13,2690,0
3,15,1299,0
4,15,20968,0
...,...,...,...
9995,46118,20106,0
9996,46124,19677,0
9997,46125,12878,0


In [6]:
a3_num = products.attribute_3.unique().size
attribute_3_dict = {}
a3_list = []
for i in range(a3_num):
    attribute_3_dict[i] = products[products.attribute_3 == i].index.to_list()
print("a3 keys ", attribute_3_dict.keys())
for i in range(a3_num):
    if len(attribute_3_dict[i]) >= 2 and len(attribute_3_dict[i]) <= 500:
        a3_list.append(i)

sim_list = []
cat_score = 0
list_of_compare = ["manufacturer_id", "attribute_1", "attribute_2", "attribute_3", "attribute_4", "attribute_5", "categories"]
for a in tqdm(a3_list, desc="Processing Groups"):
    for i in tqdm(range(len(attribute_3_dict[a])) ,leave=False, desc=f"Outer Loop {a}"):
        for j in range(i+1, len(attribute_3_dict[a])):
            sim_score = 0
            cat_score = 0
            for k in range(7):
                if products.loc[attribute_3_dict[a][i], list_of_compare[k]] == products.loc[attribute_3_dict[a][j], list_of_compare[k]] and k != 6:
                    sim_score += 1
                elif k == 6:
                    num_common = len(set(products.loc[attribute_3_dict[a][i], list_of_compare[6]]) & set(products.loc[attribute_3_dict[a][j], list_of_compare[6]]))
                    cat_score += num_common
            sim_list.append([attribute_3_dict[a][i], attribute_3_dict[a][j]])


sim_list1 = []

for i in range(len(sim_list)):
    if sim_list[i][-2] > 4 & sim_list[i][-1] > 3:
        sim_list1.append(sim_list[i])
     
print(len(sim_list1))

test_products = test_set['product_id'].to_numpy()
sim_list1_np = np.array(sim_list1)

check_list = set()
sim_check_list = []

for product_id in tqdm(test_products, desc="Processing"):
    matches = sim_list1_np[np.any(sim_list1_np == product_id, axis=1)]
    for k in matches:
        k_tuple = tuple(k)
        if k_tuple in check_list:
            sim_check_list.append(k_tuple)
        else:
            check_list.add(k_tuple)

sim_list1_copy = sim_list1.copy()
unique_checklist = []
for x in sim_check_list:
    if x not in unique_checklist:
        unique_checklist.append(x)

for i in tqdm(unique_checklist):
    sim_list1_copy.remove(list(i))

a3 keys  dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 

Processing Groups: 100%|██████████| 465/465 [01:52<00:00,  4.14it/s]


925706


100%|██████████| 67174/67174 [10:52<00:00, 102.90it/s]


In [8]:
try:
    transactions = pd.read_csv("updated_transaction.csv")
except:
    transaction_copy = transactions.copy()
    sim_list1_copy.sort(key=lambda x: x[0])

    for index, row in tqdm(test_set.iterrows()):
        product_id = row['product_id']

        if np.isin(product_id, sim_list1_copy):
            similar_product_id_list = [x[1] if x[0] == product_id else x[0] for x in sim_list1_copy if product_id in x]
            for i in similar_product_id_list:
                transaction_copy['product_id'] = transaction_copy['product_id'].replace(i, product_id)
        else:
            continue

    transaction_copy.to_csv('updated_transaction.csv', index=False)
    transactions = transaction_copy

transactions

,customer_id,product_id,purchase_date,quantity,day
0,38769,3477,2020-06-01,1,0
1,42535,30474,2020-06-01,1,0
2,42535,20494,2020-06-01,1,0
3,42535,20131,2020-06-01,1,0
4,42535,4325,2020-06-01,1,0
...,...,...,...,...,...
1071533,22114,29567,2021-01-31,2,244
1071534,22114,21068,2021-01-31,1,244
1071535,10900,18270,2021-01-31,1,244
1071536,17894,31265,2021-01-31,5,244


Method 1

In [9]:
for i in range(len(test_set)):
    cusprod = test_set.loc[i]
    cus = cusprod["customer_id"]
    prod = cusprod["product_id"]
    df = transactions[(transactions["customer_id"] == cus) & (transactions["product_id"] == prod)]
    if df[len(df)//2:].day.diff()[1:].mean() > 35:
        test_set.loc[i,"prediction"] = 0
    else:
        if len(df) <= 1:
            pred = 4
        else:
            target = math.ceil((df.iloc[-1].day - df.iloc[-2].day)/7)
            if target > 4:
                pred = 4
            else:
                pred = target
        test_set.loc[i,"prediction"] = pred

test_set.to_csv("test.csv")
test_set

,customer_id,product_id,prediction
id,,,
0,0,20664,0
1,0,28231,0
2,13,2690,0
3,15,1299,3
4,15,20968,4
...,...,...,...
9995,46118,20106,0
9996,46124,19677,4
9997,46125,12878,0


Method 2

In [10]:
sorted_transactions = transactions.sort_values(by=['customer_id', 'product_id', 'purchase_date'])
sorted_transactions['next_purchase_day'] = sorted_transactions.groupby(['customer_id', 'product_id'])['day'].shift(-1)
transactions["label"] = sorted_transactions['next_purchase_day'] - sorted_transactions['day']
transactions["label"].fillna(0, inplace=True)

nonzero_labels = transactions[transactions["label"] > 0]
nonzero_labels = nonzero_labels[nonzero_labels["label"] < 28]
nonzero_labels

C:\Users\Oğuzhan ÇELİK\AppData\Local\Temp\ipykernel_26608\2684778522.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  transactions["label"].fillna(0, inplace=True)


,customer_id,product_id,purchase_date,quantity,day,label
8,29737,11535,2020-06-01,3,0,6.0
102,7949,11251,2020-06-01,5,0,1.0
286,15979,28135,2020-06-01,1,0,15.0
300,40278,31687,2020-06-01,4,0,15.0
302,40278,16778,2020-06-01,5,0,15.0
...,...,...,...,...,...,...
1055084,30904,29794,2021-01-29,2,242,1.0
1058348,37228,27956,2021-01-30,5,243,1.0
1059315,6766,17089,2021-01-30,1,243,1.0
1059316,6766,28739,2021-01-30,1,243,1.0


In [11]:
# Feature calculation function
def calculate_features(transactions, row, products):
    customer_id = row['customer_id']
    product_id = row['product_id']
    transaction_day = row["day"]
    product = products.loc[product_id]

    # Filter past transactions for the specific customer-product pair
    past_transactions = transactions[(transactions['customer_id'] == customer_id) & (transactions['product_id'] == product_id) & (transactions['day'] < transaction_day)]
    past_customer = transactions[(transactions['customer_id'] == customer_id) & (transactions['day'] <= transaction_day)]

    days_since_last_purchase = transaction_day - past_transactions['day'].max() if not past_transactions.empty else 0
    skewness = past_transactions["day"].skew()
    if skewness == np.nan:
        skewness = 0
    return {
        'transaction_count': len(past_transactions),
        'last_quantity': row["quantity"],
        'days_since_last_purchase': days_since_last_purchase,
        'purchase_skewness': skewness,
        'customer_transaction_freq': len(past_customer)/transaction_day if transaction_day!=0 else 0,
        'product_id': product_id,
        'attribute_1': product["attribute_1"],
        'attribute_2': product["attribute_2"],
        'attribute_3': product["attribute_3"],
        'attribute_4': product["attribute_4"],
        'attribute_5': product["attribute_5"],
        'label': row["label"],
    }

In [12]:
# Apply function iteratively
features = []
for idx, row in nonzero_labels.iterrows():
    # Call the feature calculation function
    feature_dict = calculate_features(nonzero_labels, row, products)
    features.append(feature_dict)

# Create features dataframe
features_df = pd.DataFrame(features)
features_df = features_df[features_df["transaction_count"] != 0]
features_df.fillna(0, inplace=True)
features_df["label"] = np.ceil(features_df["label"]/7)

,transaction_count,last_quantity,days_since_last_purchase,purchase_skewness,customer_transaction_freq,product_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,label
286,1,1,1,0.0,1.000000,17547,10,2,536,0,66,1.0
460,2,1,1,0.0,1.000000,17547,10,2,536,0,66,1.0
470,1,2,1,0.0,1.666667,23948,4,0,321,0,117,3.0
597,1,1,5,0.0,0.400000,10721,3,3,503,0,144,4.0
686,1,4,4,0.0,1.200000,10849,10,0,348,0,95,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
27007,2,1,125,0.0,0.029412,3857,10,1,503,3,17,1.0
27019,1,3,22,0.0,0.012552,27635,4,2,487,3,81,1.0
27048,1,4,6,0.0,0.012448,22151,4,0,28,0,81,1.0
27052,1,1,166,0.0,0.024793,21671,4,0,485,3,66,1.0


In [13]:
y = features_df["label"]
X = features_df.drop(columns="label")
categorical_features = ['product_id', 'attribute_1', "attribute_2", "attribute_3", "attribute_4", "attribute_5"]
train_data = Pool(data=X, label=y, cat_features=categorical_features)

model = CatBoostClassifier(verbose=10)

# Train the model
model.fit(train_data)

# Make Predictions
predictions = model.predict(X)
print(accuracy_score(y, predictions))
print(model.feature_importances_)

Learning rate set to 0.08796
0:	learn: 1.3683866	total: 207ms	remaining: 3m 26s
10:	learn: 1.2706651	total: 872ms	remaining: 1m 18s
20:	learn: 1.2347697	total: 1.57s	remaining: 1m 13s
30:	learn: 1.2183220	total: 2.29s	remaining: 1m 11s
40:	learn: 1.2091139	total: 3.02s	remaining: 1m 10s
50:	learn: 1.2003261	total: 3.71s	remaining: 1m 9s
60:	learn: 1.1928671	total: 4.43s	remaining: 1m 8s
70:	learn: 1.1861367	total: 5.11s	remaining: 1m 6s
80:	learn: 1.1790173	total: 5.87s	remaining: 1m 6s
90:	learn: 1.1748881	total: 6.5s	remaining: 1m 4s
100:	learn: 1.1685561	total: 7.19s	remaining: 1m 4s
110:	learn: 1.1624642	total: 7.86s	remaining: 1m 2s
120:	learn: 1.1559773	total: 8.65s	remaining: 1m 2s
130:	learn: 1.1488591	total: 9.36s	remaining: 1m 2s
140:	learn: 1.1429847	total: 10.1s	remaining: 1m 1s
150:	learn: 1.1369996	total: 10.8s	remaining: 1m
160:	learn: 1.1310751	total: 11.4s	remaining: 59.6s
170:	learn: 1.1237646	total: 12.1s	remaining: 58.5s
180:	learn: 1.1182961	total: 12.6s	remaining:

In [14]:
test_features = []
test_copy = test_set.copy()
for idx, _row in tqdm(test_copy.iterrows()):
    product_id = _row['product_id']
    row = transactions[(transactions["customer_id"] == _row["customer_id"]) & (transactions["product_id"] == product_id)].iloc[-1]
    similar_product_id_list = [x[1] if x[0] == product_id else x[0] for x in sim_list1_kopya if product_id in x]
    for i in similar_product_id_list:
        test_copy['product_id'] = test_copy['product_id'].replace(i, product_id)
    # Call the feature calculation function
    feature_dict = calculate_features(transactions, row, products)
    test_features.append(feature_dict)

# Create features dataframe
test_features_df = pd.DataFrame(test_features)
test_features_df.fillna(0, inplace=True)

display(test_features_df)

predictions = model.predict(test_features_df.drop(columns="label"))

10000it [09:26, 17.65it/s]


,transaction_count,last_quantity,days_since_last_purchase,purchase_skewness,customer_transaction_freq,product_id,attribute_1,attribute_2,attribute_3,attribute_4,attribute_5,label
0,2,1,63,0.000000,0.072539,20664,4,0,284,0,66,0.0
1,2,2,45,0.000000,0.075630,28231,4,3,468,3,108,0.0
2,3,1,55,-0.204892,0.218447,2690,4,3,491,0,66,0.0
3,3,1,19,-1.293343,0.189427,1299,4,0,474,0,108,0.0
4,3,1,30,-0.893683,0.205882,20968,4,0,444,0,144,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4,1,41,-0.464070,0.200000,20106,4,0,491,0,66,0.0
9996,5,1,29,-0.153076,0.131356,19677,4,0,491,3,154,0.0
9997,2,1,77,0.000000,0.065574,12878,4,0,491,0,66,0.0
9998,4,1,15,0.493853,0.127551,7963,4,0,485,3,154,0.0


In [16]:
for i in range(len(test_set)):
    cusprod = test_set.loc[i]
    cus = cusprod["customer_id"]
    prod = cusprod["product_id"]
    df = transactions[(transactions["customer_id"] == cus) & (transactions["product_id"] == prod)]
    if df[len(df)//2:].day.diff()[1:].mean() > 35:
        test_set.loc[i,"prediction"] = 0
    else:
        test_set.loc[i,"prediction"] = predictions[i]

test_set.to_csv("test.csv")
test_set

,customer_id,product_id,prediction
id,,,
0,0,20664,0
1,0,28231,0
2,13,2690,0
3,15,1299,3
4,15,20968,3
...,...,...,...
9995,46118,20106,0
9996,46124,19677,4
9997,46125,12878,0
